In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
from deepdrr.utils import test_utils
from deepdrr.frontend.frontend import *
from deepdrr.frontend.renderer import *
from deepdrr.frontend.profiles import *


In [14]:
f = test_utils.download_sampledata("CT-chest")
mf = Path("tests/resources/suzanne.stl")
mf2 = Path("tests/resources/threads.stl")
print(f)

Using downloaded and verified file: /home/pelvisvr/datasets/DeepDRR_DATA/CT-chest.nrrd
/home/pelvisvr/datasets/DeepDRR_DATA/CT-chest.nrrd


In [15]:
volume = Volume.from_nrrd(f)

mesh1 = Mesh.from_stl(mf, tag="mesh1")
mesh2 = Mesh.from_stl(mf2, tag="mesh2")

carm = MobileCArm()

graph = TransformTree()
graph.add(carm)
graph.add(volume)
graph.add(mesh1)
graph.add(mesh2)

scene = GraphScene(graph)

drr_json_path = Path("drr.json")

drr_settings = DRRRenderSettings(
    width=512,
    height=512,
    neglog=True,
)
drr_prof = DRRRenderProfile(
    settings=drr_settings,
    renderer=DeferredRenderer(drr_json_path),
    # renderer=SynchronousRenderer(),
    scene=scene
)

rast_settings = RasterizeRenderSettings(
    width=512,
    height=512,
    max_mesh_hits=32,
)
rasterize_prof = RasterizeRenderProfile(
    settings=rast_settings,
    renderer=SynchronousRenderer(),
    scene=scene
)



In [23]:

mesh1.world_from_anatomical = geo.FrameTransform.from_translation([-30, 50, 200])


with drr_prof, rasterize_prof:
    for frame_idx in range(10):
        mesh2.world_from_anatomical = geo.FrameTransform.from_translation([0, 0, 10*frame_idx])

        carm.move_by(
            delta_isocenter=[0, 0, 10],
            delta_alpha=5*frame_idx,
            delta_beta=0,
            delta_gamma=0,
            degrees=False,
        )

        drr = drr_prof.render_drr()

        segs = rasterize_prof.render_seg()
        hits = rasterize_prof.render_hits()


    

In [22]:
a = Primitive(primitive_id="a", data=StlMesh(url="file:///home/pelvisvr/datasets/DeepDRR_DATA/CT-chest.nrrd", material_name="material_name", density=1.0, priority=1, addtive=True, subtractive=True))
b = Primitive(primitive_id="b", data=H5Volume(url="file:///home/pelvisvr/datasets/DeepDRR_DATA/CT-chest.nrrd", priority=1))

cam = Camera(transform=Matrix4x4(data=np.eye(4).flatten()), intrinsic=RenderCameraIntrinsic(fx=1.0, fy=1.0, cx=0.0, cy=0.0, near=1.0, far=100.0))
inst = Instance(primitive_id="a", transform=Matrix4x4(data=np.eye(4).flatten()), morph_weights=[1.0, 2.0, 3.0])
frame = Frame(camera=cam, instances=[inst])

TypeError: StlMesh.__init__() missing 1 required positional argument: 'path'

In [ ]:
c = Scene(primitives=[a,b], frames=[frame])

In [ ]:
# save to asdf.json

with open("asdf.json", "w") as f:
    f.write(c.model_dump_json(indent=2))

In [ ]:
# load
with open("asdf.json", "r") as f:
    d = Scene.model_validate_json(f.read())

d

Scene(primitives=[Primitive(primitive_id='a', data=StlMesh(prim_data_type='StlMesh', url=Url('file:///home/pelvisvr/datasets/DeepDRR_DATA/CT-chest.nrrd'), material_name='material_name', density=1.0, priority=1, addtive=True, subtractive=True)), Primitive(primitive_id='b', data=H5Volume(prim_data_type='H5Volume', url=Url('file:///home/pelvisvr/datasets/DeepDRR_DATA/CT-chest.nrrd'), priority=1))], frames=[Frame(camera=Camera(transform=Matrix4x4(data=[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0]), intrinsic=CameraIntrinsic(fx=1.0, fy=1.0, cx=0.0, cy=0.0, near=1.0, far=100.0)), instances=[Instance(primitive_id='a', transform=Matrix4x4(data=[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0]), morph_weights=[1.0, 2.0, 3.0])])])

In [ ]:
a.data.url.path

'/home/pelvisvr/datasets/DeepDRR_DATA/CT-chest.nrrd'

In [ ]:
import urllib.request

o = urllib.request.urlopen("file:///home/pelvisvr/datasets/DeepDRR_DATA/CT-chest.nrrd")

asdf = o.read()

print(len(asdf) / 1e6)

42.189103


In [ ]:
type(asdf)

bytes